# TensorFlow Self-define自定义

## 自定义loss函数
`tf.reduce_mean(tf.losses.softmax_cross_entropy(y, y_))`和`tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_), axis=-1))`是等价的<br>
tensorflow比较底层，loss函数定义较为自由

## 自定义layer
自定义网络可以通过`tf.Variable`实现，所有网络都是基于全相连网络实现（就是矩阵乘发。本质上是对特征向量维度的任意改变）
```
weights = tf.Variable(tf.random_normal([input_size, hidden_size]), name='kernel')
bias = tf.Variable(tf.random_normal([hidden_size]), name='bias')
linear1 = tf.matmul(x, weights) + bias
```
和<br>
```
tf.layers.dense(inputs=linear1, units=num_classes)
```
等价

## 自定义op
一般的操作可以由tensorflow自带的运算组合而成<br>
若需要自定义特殊运算，需要用到C/C++，难度较大。网络上存在这样的案例，有兴趣的读者可以尝试<br>

## 自定义optimizer
tensorflow中对optimizer的定义较难。笔者以SGD的源码（部分，因为很多其他内容不常用）为例
```
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from tensorflow.python.training import optimizer
from tensorflow.python.training import training_ops

class GradientDescentOptimizer(optimizer.Optimizer):
    def __init__(self, learning_rate, use_locking=False, name="GradientDescent"):
        super(GradientDescentOptimizer, self).__init__(use_locking, name)
        self._learning_rate = learning_rate

    def _apply_dense(self, grad, var):
        return training_ops.apply_gradient_descent(
            var,
            math_ops.cast(self._learning_rate_tensor, var.dtype.base_dtype),
            grad,
            use_locking=self._use_locking).op

    def _prepare(self):
        self._learning_rate_tensor = ops.convert_to_tensor(self._learning_rate,
                                                           name="learning_rate")
```

`__init__`，`_apply_dense`，`_prepare`是三个最基础的必要函数，分别用于：定义，梯度计算和参数类型转换。
在`_apply_dense`中，又将参数送到`apply_gradient_descent`中；而`apply_gradient_descent`有利用tensorflow写好的`_op_def_lib.apply_op("ApplyGradientDescent", ...)`进行操作
对于想自定义optimizer的用户来说，并不友好


实际上，TF中的优化器已足够用户使用，无需额外定义。但是在某些情况下，我们需要修改网络的梯度，可以采用如下方式：<br>
``
optimizer_op = tf.train.AdamOptimizer()
gradients, variables = zip(*optimizer_op.compute_gradients(loss_op))
-# some operations on gradients, like
gradients, _ = tf.clip_by_value(gradients, 5.0)
train_op = optimizer_op.apply_gradients(zip(gradients, variables))
``